In [26]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets.cifar10 import load_data

In [29]:
import numpy as np
def make_val(train_data, val_size):
    '''
    Parameter
    train_data : train data tuple (e.g. (train_features, train_labels))
    val_num :  size of validation data set

    Return
    train_data : train data tuple (e.g. (train_features, train_labels))
    val_data : validation data tuple(e.g. (val_features, val_labels))

    Example
    train_data, val_data = make_val(train_data, 2000)
    '''
    val_idx = np.random.choice(range(len(train_data[0])), val_size, replace=False)
    val_data = (train_data[0][val_idx], train_data[1][val_idx])
    train_data = (np.delete(train_data[0], val_idx, 0), np.delete(train_data[1], val_idx, 0))

    return train_data, val_data

def make_onehot(dataset, cate_num):
    '''
    Parameter
    dataset : data tuple (e.g. (train_features, train_labels))
    cate_num : len of one-hot vector (e.g. cifar10's label: 10)

    Return
    dataset : data tuple with one-hot label

    Example
    train_data = make_onehot(train_data, 10)
    '''
    dataset = (dataset[0], np.squeeze(np.eye(cate_num)[dataset[1]]))

    return dataset


train_data, test_data = cifar10.load_data()

train_data, val_data = make_val(train_data, 2000)

train_data = make_onehot(train_data, 10) # Shape: ((48000, 32, 32, 3), (48000, 10))
val_data = make_onehot(val_data, 10) # Shape: ((2000, 32, 32, 3), (2000, 10))
test_data = make_onehot(test_data, 10) # Shape: ((10000, 32, 32, 3), (10000, 10))


# parameters
learning_rate = 0.001
training_epochs = 10
BATCH_SIZE = 32
batch_size = tf.placeholder(tf.int64)
total_batch = len(train_data[0]) // BATCH_SIZE

# create the Dataset
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])

dataset = tf.data.Dataset.from_tensor_slices((X, Y)).shuffle(buffer_size=10000).batch(batch_size).repeat()

# create the iter
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

AttributeError: module 'tensorflow' has no attribute 'data'

In [ ]:
# CIFAR-10 데이터를 다운로드하고 데이터를 불러옵니다.
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
def normalize(x):
    return x / 255

In [ ]:
x_train = list(map(normalize, x_train))
x_test = list(map(normalize, x_test))

In [ ]:
k = [np.eye(10)[i] for i in y_train]
y_train = [k[i][0] for i in range(len(k))]

k = [np.eye(10)[i] for i in y_test]
y_test = [k[i][0] for i in range(len(k))]

In [ ]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [2]:
# parameters
training_epochs = 2000
batch_size = 64

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])

W1 = tf.Variable(tf.random_normal([3, 3, 3, 64], stddev=0.05))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')+b1
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.05))
b2 = tf.Variable(tf.random_normal([128]))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')+b2
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.05))
b3 = tf.Variable(tf.random_normal([256]))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')+b3
L3 = tf.nn.relu(L3)
L3_flat = tf.reshape(L3, [-1, 256 * 8 * 8])

W4 = tf.get_variable("W4", shape=[256 * 8 * 8, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1024]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)


W5 = tf.get_variable("W5", shape=[1024, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4, W5) + b5

In [4]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer().minimize(cost)
#AdamOptimizer
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [6]:
for epoch in range(training_epochs):
    batch = next_batch(batch_size, x_train, y_train)
    sess.run(optimizer, feed_dict={X: batch[0], Y: batch[1]})
        
    if epoch % 10 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: batch[0], Y: batch[1]})
        print("Step " + str(epoch) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))
print("Optimization Finished!")

NameError: name 'next_batch' is not defined

In [26]:
# Test model and check accuracy
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_test, Y: y_test }))

Accuracy: 0.6477


In [23]:
import sys
sys.version

'3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 12:04:33) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [30]:
import tensorflow as tf

In [31]:
from tensorflow.python.keras import layers

ModuleNotFoundError: No module named 'tensorflow.python.keras'